In [ ]:
from hashlib import sha256

In [ ]:
encrypted_elements = [b'217',b'354',b'466']

## Simple Bloom Encoding Example

In [ ]:
num_bits = 32
num_hashfunctions = 4
filterlist = ['0'] * num_bits

for element in encrypted_elements:
    element1 = str.encode('1') + element
    k = sha256(element1).hexdigest()
    h1 = int(k,16) % num_bits

    element2 = str.encode('2') + element
    k = sha256(element2).hexdigest()
    h2 = int(k,16) % num_bits
    
    for i in range(num_hashfunctions):
        pos = ((h1 + i * h2) % num_bits)
        filterlist[num_bits-1-pos]='1'
        
filterstring = ''.join(filterlist)

In [ ]:
filterstring

In [ ]:
for number in range(50):
    check = True

    element1 = str.encode('1') + str.encode('{0:d}'.format(number))
    k = sha256(element1).hexdigest()
    h1 = int(k,16) % num_bits

    element2 = str.encode('2') + str.encode('{0:d}'.format(number))
    k = sha256(element2).hexdigest()
    h2 = int(k,16) % num_bits
    
    for i in range(num_hashfunctions):
        pos = ((h1 + i * h2) % num_bits)
        if (filterlist[num_bits-1-pos]=='1'):
            check = check & True
        else:
            check = False
    if check==True:
        print(number)

## Simple GCS Encoding Example

In [ ]:

gcshash_range = 128
gcsdiv = 5

In [ ]:
# For all encrypted elements, calculate hash and then bucket value
ulist = []
for element in encrypted_elements:
    k = sha256(element).hexdigest()
    ks = int(k,16) % gcshash_range
    print(ks, k)
    ulist.append(ks)

# Sort the hash bucket values
ulist.sort()
# Calculate deltas between sorted hash bucket values 
udiff = [ulist[0]] + [ulist[n]-ulist[n-1] for n in range(1,len(ulist))]

In [ ]:
ulist

In [ ]:
udiff

In [ ]:
# For all delta hash bucket values encode as unary portion for quotient followed by binary for remainder.
# Pad with leading zeros so binary portion is of consistent length.
# Concatenate with previous values

encoded = ''
for diff in udiff:
    if diff != 0:
        quot = int(diff / pow(2,gcsdiv))
        rem = diff % pow(2,gcsdiv)
        next = '{0:b}'.format(rem) + '1' + ('0' * quot)
        pad = next.zfill(quot+gcsdiv+1)
        print(quot,rem, next, pad)
        encoded = pad + encoded

In [ ]:
encoded